In [1]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sandeepsangole/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import checklist
import spacy
import itertools

import numpy as np
import torch
import transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

from checklist.test_suite import TestSuite
import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb
from transformers import pipeline

from datasets import Dataset
import datasets
squad_adv_addSent = datasets.load_dataset('squad_adversarial', 'AddOneSent')
nq_data = datasets.load_dataset("json",data_files="./fp-dataset-artifacts-main/datasets/nq_train.json", field='data')
squad = datasets.load_dataset('squad')
squad_v2 = datasets.load_dataset('squad_v2')


editor = checklist.editor.Editor()
editor.tg

In [4]:

model_dir = "./fp-dataset-artifacts-main/squad_trained_model/"
model_nlp = pipeline('question-answering', model=AutoModelForQuestionAnswering.from_pretrained(model_dir), tokenizer=AutoTokenizer.from_pretrained(model_dir))

In [12]:
## Vocabulary 
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]
print(adj)

[('old', 'old'), ('smart', 'smart'), ('tall', 'tall'), ('young', 'young'), ('strong', 'strong'), ('short', 'short'), ('tough', 'tough'), ('cool', 'cool'), ('fast', 'fast'), ('nic', 'nice'), ('small', 'small'), ('dark', 'dark'), ('wis', 'wise'), ('rich', 'rich'), ('great', 'great'), ('weak', 'weak'), ('high', 'high'), ('slow', 'slow'), ('strang', 'strange'), ('clean', 'clean')]


In [13]:
from checklist.pred_wrapper import PredictorWrapper
pred = lambda x: ([model_nlp({"context":x[0], "question":x[1]})['answer']], [model_nlp({"context":x[0], "question":x[1]})['score']])
testing_example = [("Samantha is slower than Jordan.","Who is less slow?")]

def predict_and_score_fun(input):
    
    preds = []
    scores = []
    for i in input:
      context = i[0] 
      question = i[1]
      pred = model_nlp({"context":context,"question":question})
      preds.append(pred['answer'])
      scores.append(pred['score'])
    return (preds, scores)

new_pred = PredictorWrapper.wrap_predict(predict_and_score_fun)

print(predict_and_score_fun(testing_example))

(['Samantha'], [0.735386073589325])


In [14]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

def checklist_to_squad(name, new_data, current_data=None, num_samples=None):

  ids = []
  titles = []
  contexts = []
  questions = []
  answers = []

  for example_group_idx in range(len(new_data.data)):
    for example_idx in range(len(new_data.data[example_group_idx])):
      #print("our indices: ", example_group_idx, example_idx, "in ", len(new_data.data), len(example))
      example_data = new_data.data[example_group_idx][example_idx]
      context, question = example_data
      example_label = new_data.labels[example_group_idx][example_idx]
      ids.append("a"+str(example_group_idx)+str(example_idx))
      titles.append("name")
      contexts.append(context)
      questions.append(question)

      answers.append({'text':[example_label], "answer_start":[context.find(example_label)]})

  my_data = {
      "id": ids,
      "title": titles,
      "context": contexts,
      "question": questions,
      "answers": answers
    
  }
  checklist_data = Dataset.from_dict(my_data, features=squad["train"].features)
  if num_samples != None:
    selection = random.sample(range(checklist_data.num_rows), num_samples)
    checklist_data = checklist_data.select(selection)
  if current_data == None:
    return checklist_data
  else:
    return datasets.concatenate_datasets([current_data, checklist_data])

#checklist_data = checklist_to_squad("Profession_vs_nationality", t)
#print(checklist_data)

In [15]:
suite = TestSuite()

In [17]:
t = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    ),(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    )
    ],
    labels = ['{first_name1}','{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=50,
    save=True
    )
name = 'A is COMP than B. Who is more / less COMP?'
description = ''
test = MFT(**t, name=name, description=description, capability='Vocabulary')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("more_less_comp", t)
checklist_data_full = checklist_to_squad("more_less_comp", t)


Predicting 98 examples
Test cases:      49
Fails (rate):    48 (98.0%)

Example fails:
C: Nicole is smarter than Lucy.
Q: Who is less smart?
A: Lucy
P: Nicole


----
C: Charlie is weaker than Helen.
Q: Who is less weak?
A: Helen
P: Charlie


----
C: Leslie is slower than Harriet.
Q: Who is less slow?
A: Harriet
P: Leslie


----


In [18]:
checklist_data[10]

{'id': 'a50',
 'title': 'name',
 'context': 'Louise is shorter than Roger.',
 'question': 'Who is less short?',
 'answers': {'text': ['Roger'], 'answer_start': [23]}}

In [29]:
import numpy as np

np.unique(squad['train']['title'])

array(['2008_Sichuan_earthquake', '2008_Summer_Olympics_torch_relay',
       '51st_state', 'ASCII', 'A_cappella', 'Adolescence',
       'Adult_contemporary_music',
       'Affirmative_action_in_the_United_States', 'Age_of_Enlightenment',
       'Aircraft_carrier', 'Airport', 'Alaska', 'Alexander_Graham_Bell',
       'Alfred_North_Whitehead', 'Alloy', 'Alps', 'Alsace',
       'American_Idol', 'Animal', 'Ann_Arbor,_Michigan', 'Annelid',
       'Antarctica', 'Antenna_(radio)', 'Anthropology',
       'Anti-aircraft_warfare', 'Antibiotics', 'Apollo',
       'Appalachian_Mountains', 'Architecture', 'Arena_Football_League',
       'Armenia', 'Armenians', 'Arnold_Schwarzenegger', 'Arsenal_F.C.',
       'Ashkenazi_Jews', 'Asphalt', 'Aspirated_consonant',
       'Association_football', 'Asthma', 'Athanasius_of_Alexandria',
       'Atlantic_City,_New_Jersey', 'Avicenna', 'BBC_Television',
       'Bacteria', 'Baptists', 'Beer',
       'BeiDou_Navigation_Satellite_System', 'Bermuda', 'Bern', 'Beyon

In [113]:
def crossproduct(t):
    # takes the output of editor.template and does the cross product of contexts and qas
    ret = []
    ret_labels = []
    for x in t.data:
        cs = x['contexts']
        qas = x['qas']
        d = list(itertools.product(cs, qas))
        ret.append([(x[0], x[1][0]) for x in d])
        ret_labels.append([x[1][1] for x in d])
    t.data = ret
    t.labels = ret_labels
    return t

def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')

def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)

state = editor.suggest('John is very {mask} about the project.')[:20]
print(', '.join(editor.suggest('John is {mask} {state} about the project.', state=state)[:30]))
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

very, pretty, extremely, also, still, quite, more, really, not, clearly, fairly, incredibly, particularly, now, understandably, rather, cautiously, surprisingly, certainly, feeling, so, especially, definitely, generally, most, highly, super, reportedly, being, obviously


In [114]:
import random

In [115]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=50,
    save=True
    ))
name = 'Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?'
desc = ''
test = MFT(**t, name=name, description=desc, capability='Vocabulary')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("intensifiers_and_reducers", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("intensifiers_and_reducers", t, checklist_data_full)

Predicting 600 examples
Test cases:      50
Fails (rate):    50 (100.0%)

Example fails:
C: Nick is passionate about the project. Paul is somewhat passionate about the project.
Q: Who is least passionate about the project?
A: Paul
P: Nick is passionate about the project. Paul

C: Paul is somewhat passionate about the project. Nick is passionate about the project.
Q: Who is least passionate about the project?
A: Paul
P: Nick

C: Paul is passionate about the project. Nick is really passionate about the project.
Q: Who is most passionate about the project?
A: Nick
P: Paul is passionate about the project. Nick


----
C: Victoria is highly excited about the project. Dan is excited about the project.
Q: Who is most excited about the project?
A: Victoria
P: Dan

C: Victoria is excited about the project. Dan is mildly excited about the project.
Q: Who is most excited about the project?
A: Victoria
P: Dan

C: Dan is mildly excited about the project. Victoria is excited about the project.
Q: Who

In [116]:
checklist_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 200
})

In [117]:
def crossproduct(t):
    # takes the output of editor.template and does the cross product of contexts and qas
    ret = []
    ret_labels = []
    for x in t.data:
        cs = x['contexts']
        qas = x['qas']
        d = list(itertools.product(cs, qas))
        ret.append([(x[0], x[1][0]) for x in d])
        ret_labels.append([x[1][1] for x in d])
    t.data = ret
    t.labels = ret_labels
    return t

def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')

def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)

state = editor.suggest('John is very {mask} about the project.')[:20]
print(', '.join(editor.suggest('John is {mask} {state} about the project.', state=state)[:30]))
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

very, pretty, extremely, also, still, quite, more, really, not, clearly, fairly, incredibly, particularly, now, understandably, rather, cautiously, surprisingly, certainly, feeling, so, especially, definitely, generally, most, highly, super, reportedly, being, obviously


In [118]:
model_nlp({'context': "Samuel is very particular about the project. Keith is somewhat particular about the project.", "question":"Who is most particular about the project?"})

{'score': 0.3198288381099701, 'start': 45, 'end': 50, 'answer': 'Keith'}

In [119]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=50,
    save=True
    ))
name = 'Animal vs Vehicle'
test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("animal_vs_vehicle", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("animal_vs_vehicle", t, checklist_data_full)

Predicting 200 examples
Test cases:      50
Fails (rate):    27 (54.0%)

Example fails:
C: Joan has a serpent and a train.
Q: What vehicle does Joan have?
A: train
P: serpent and a train


----
C: Christopher has a cat and a car.
Q: What vehicle does Christopher have?
A: car
P: a cat

C: Christopher has a car and a cat.
Q: What vehicle does Christopher have?
A: car
P: car and a cat


----
C: Don has a rabbit and a minivan.
Q: What vehicle does Don have?
A: minivan
P: rabbit and a minivan


----


In [120]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True,
    ))
name = 'Profession vs nationality'
test = MFT(**t, name=name, expect=expect_squad, description='',  capability='Taxonomy')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("profession_vs_nationality", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("profession_vs_nationality", t, checklist_data_full)

Predicting 5000 examples
Test cases:      500
Fails (rate):    402 (80.4%)

Example fails:
C: Dave is a Japanese entrepreneur.
Q: What is Dave's job?
A: entrepreneur
P: Japanese entrepreneur


----
C: Leslie is a Brazilian entrepreneur.
Q: What is Leslie's job?
A: entrepreneur
P: Brazilian entrepreneur


----
C: Judy is a Chinese agent.
Q: What is Judy's job?
A: agent
P: Chinese agent

C: Judy is an agent and Chinese.
Q: What is Judy's job?
A: agent
P: agent and Chinese


----


In [131]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))#list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )
            
        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=50,
    save=True
    ))
name = 'A is COMP than B. Who is antonym(COMP)? B'
test = MFT(**t, name=name, description='', capability='Taxonomy')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("comp_pairs", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("comp_pairs", t, checklist_data_full)

Predicting 200 examples
Test cases:      50
Fails (rate):    50 (100.0%)

Example fails:
C: Nick is sadder than Betty.
Q: Who is happier?
A: Betty
P: Nick

C: Betty is happier than Nick.
Q: Who is sadder?
A: Nick
P: Betty


----
C: Karen is louder than Ray.
Q: Who is quieter?
A: Ray
P: Karen


----
C: Scott is bigger than Kelly.
Q: Who is bigger?
A: Scott
P: Kelly

C: Kelly is smaller than Scott.
Q: Who is bigger?
A: Scott
P: Kelly


----


In [132]:
model_nlp({'context': "Maria is happier than Carol.", "question":"Who is sadder?"})

{'score': 0.6608907580375671, 'start': 0, 'end': 5, 'answer': 'Maria'}

In [133]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [134]:
import re
def change_thing(change_fn):
    def change_both(cq, **kwargs):
        context, question = cq
        a = change_fn(context, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append((c, new_q))
        return ret, meta
    return change_both
            

In [135]:
def expect_same(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    if not meta:
        return pred == orig_pred
    return pred == re.sub(r'\b%s\b' % re.escape(meta[0]), meta[1], orig_pred)

def format_replace(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

def format_replace_context(x, pred, conf, label=None, meta=None):
    ret = format_squad_with_context(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

In [136]:
!pip install jsonlines
import jsonlines
import pandas as pd

In [137]:
ls

FalsePrediction.txt        peer-assessment.pdf
Untitled.ipynb             project-ex-1.pdf
fp-dataset-artifacts-main/ project-ex-2.pdf
fp.pdf


In [138]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df = pd.DataFrame(columns=columns)

with jsonlines.open('fp-dataset-artifacts-main/baseline_eval/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df.loc[len(answers_df)] = entry
#     i+=1

In [139]:
answers_df[['title','exact_match']].groupby('title').agg({i:'value_counts' for i in answers_df[['title','exact_match']].columns[1:]}).groupby(level=0).transform(lambda x: x.div(x.sum()))


exact_match
title                         exact_match             
1973_oil_crisis               True            0.839623
                              False           0.160377
Amazon_rainforest             True            0.841530
                              False           0.158470
American_Broadcasting_Company True            0.869779
...                                                ...
Victoria_and_Albert_Museum    False           0.186441
Warsaw                        True            0.846154
                              False           0.153846
Yuan_dynasty                  True            0.785088
                              False           0.214912

[96 rows x 1 columns]

In [140]:
pd.set_option('max_colwidth', 400)


answers_df[['pred','question','answers_written','exact_match']].loc[answers_df.title == 'Packet_switching']

,pred,question,answers_written,exact_match
4807,Distributed Adaptive Message Block Switching,What did Paul Baran develop,"[Paul Baran developed the concept Distributed Adaptive Message Block Switching, the concept Distributed Adaptive Message Block Switching, Distributed Adaptive Message Block Switching]",True
4808,"provide a fault-tolerant, efficient routing method for telecommunication messages",What did Distributed Adaptive Message Block Switching do,"[provide a fault-tolerant, efficient routing method for telecommunication messages, provide a fault-tolerant, efficient routing method for telecommunication messages, fault-tolerant, efficient routing method]",True
4809,established principles of pre-allocation of network bandwidth,What did this concept contradict,"[This concept contrasted and contradicted the theretofore established principles of pre-allocation of network bandwidth, theretofore established principles of pre-allocation of network bandwidth, principles of pre-allocation of network bandwidth]",False
4810,coining the modern name packet switching,What is Donald Davies credited with,"[Davies is credited with coining the modern name packet switching and inspiring numerous packet switching networks in Europe, coining the modern name packet switching and inspiring numerous packet switching networks, coining the modern name packet switching]",True
4811,Distributed Adaptive Message Block Switching,What did Paul Baran develop in the late 1950's,"[the concept Distributed Adaptive Message Block Switching, the concept Distributed Adaptive Message Block Switching, Distributed Adaptive Message Block Switching]",True
...,...,...,...,...
4908,Very high-speed Backbone Network Service,what does vBNS stand for,"[The Very high-speed Backbone Network Service, Very high-speed Backbone Network Service, Very high-speed Backbone Network Service]",True
4909,provide high-speed interconnection between NSF-sponsored supercomputing centers and select access points in the United States,What did vBNS do,"[provide high-speed interconnection between NSF-sponsored supercomputing centers and select access points in the United States, provide high-speed interconnection between NSF-sponsored supercomputing centers and select access points in the United States, high-speed interconnection]",True
4910,MCI Telecommunications,Who operated the vBSN network,"[The network was engineered and operated by MCI Telecommunications under a cooperative agreement with the NSF, MCI Telecommunications, National Science Foundation]",True
4911,more than 100,By 199 how many universities were connected,"[By 1998, the vBNS had grown to connect more than 100 universities and research and engineering institutions via 12 national points of presence with DS-3, more than 100 universities, 100]",False


In [141]:
answers_df[answers_df['question'].str.contains("Who")].groupby('exact_match').count() # 1040 / 1240 = 83.8%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,200,200,200,200,200,200,200
True,1040,1040,1040,1040,1040,1040,1040


In [142]:
answers_df[answers_df['question'].str.contains("Which")].groupby('exact_match').count() # 353/454 = 77.75%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,101,101,101,101,101,101,101
True,353,353,353,353,353,353,353


In [143]:
answers_df[answers_df['question'].str.contains("Why")].groupby('exact_match').count() # 88/150 = 58.66666%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,62,62,62,62,62,62,62
True,88,88,88,88,88,88,88


In [144]:
answers_df[answers_df['question'].str.contains("Why")].groupby('exact_match').count() # 88/150 = 58.66666%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,62,62,62,62,62,62,62
True,88,88,88,88,88,88,88


In [145]:
answers_df[answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() <= 0.25].groupby('exact_match').count() # 2999 / 3710 = 80.8%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,711,711,711,711,711,711,711
True,2999,2999,2999,2999,2999,2999,2999


In [146]:
answers_df[answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() >= 0.75].groupby('exact_match').count() # 1556 / 2014 = 77.26%


,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,458,458,458,458,458,458,458
True,1556,1556,1556,1556,1556,1556,1556


In [147]:
answers_df[(answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() > 0.25) & (answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() < 0.75)].groupby('exact_match').count() # 3770 / 4846 = 77.80%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,1076,1076,1076,1076,1076,1076,1076
True,3770,3770,3770,3770,3770,3770,3770


In [148]:
mask = answers_df['question'].str.contains(r'[0-9]')
answers_df[mask].groupby('exact_match').count() # 1035 / 1301 = 79.55%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,266,266,266,266,266,266,266
True,1035,1035,1035,1035,1035,1035,1035


In [149]:
answers_df[~mask].groupby('exact_match').count() # 7290 / 9269 = 78.65%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,1979,1979,1979,1979,1979,1979,1979
True,7290,7290,7290,7290,7290,7290,7290


In [150]:
answers_df.groupby('exact_match').count()

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,2245,2245,2245,2245,2245,2245,2245
True,8325,8325,8325,8325,8325,8325,8325


In [151]:
answers_df.groupby('correct').count()

KeyError: 'correct'

In [152]:
answers_df.loc[(answers_df.question.str.contains("Why"))]

,id,title,context,pred,question,answers_written,answers_idx,exact_match
824,57337ddc4776f41900660bbe,Warsaw,"There are 13 natural reserves in Warsaw – among others, Bielany Forest, Kabaty Woods, Czerniaków Lake. About 15 kilometres (9 miles) from Warsaw, the Vistula river's environment changes strikingly and features a perfectly preserved ecosystem, with a habitat of animals that includes the otter, beaver and hundreds of bird species. There are also several lakes in Warsaw – mainly the oxbow lakes, ...",to clean them of plants and sediments,Why are the small lakes in the parks emptied before winter?,"[to clean them, clean them, to clean them of plants and sediments]","[590, 593, 590]",True
842,573399b54776f41900660e66,Warsaw,"Like many cities in Central and Eastern Europe, infrastructure in Warsaw suffered considerably during its time as an Eastern Bloc economy – though it is worth mentioning that the initial Three-Year Plan to rebuild Poland (especially Warsaw) was a major success, but what followed was very much the opposite. However, over the past decade Warsaw has seen many improvements due to solid economic gr...",solid economic growth,Why has Warsaw seen many improvements over the past decade?,"[solid economic growth, solid economic growth, economic growth]","[379, 379, 385]",True
883,5733a32bd058e614000b5f36,Warsaw,"Their local rivals, Polonia Warsaw, have significantly fewer supporters, yet they managed to win Ekstraklasa Championship in 2000. They also won the country’s championship in 1946, and won the cup twice as well. Polonia's home venue is located at Konwiktorska Street, a ten-minute walk north from the Old Town. Polonia was relegated from the country's top flight in 2013 because of their disastro...",their disastrous financial situation,Why was Polonia relegated from the country's top flight in 2013?,"[disastrous financial situation, their disastrous financial situation, disastrous financial situation.]","[388, 382, 388]",True
908,5733140a4776f419006606e4,Warsaw,"The first historical reference to Warsaw dates back to the year 1313, at a time when Kraków served as the Polish capital city. Due to its central location between the Polish–Lithuanian Commonwealth's capitals of Kraków and Vilnius, Warsaw became the capital of the Commonwealth and of the Crown of the Kingdom of Poland when King Sigismund III Vasa moved his court from Kraków to Warsaw in 1596. ...","it has survived many wars, conflicts and invasions throughout its long history","Why did Warsaw gain the title of the ""Phoenix City""?","[survived many wars, conflicts and invasions, it has survived many wars, conflicts and invasions, because it has survived many wars, conflicts and invasions throughout its long history]","[1250, 1243, 1235]",False
927,5733266d4776f41900660715,Warsaw,"In 1529, Warsaw for the first time became the seat of the General Sejm, permanent from 1569. In 1573 the city gave its name to the Warsaw Confederation, formally establishing religious freedom in the Polish–Lithuanian Commonwealth. Due to its central location between the Commonwealth's capitals of Kraków and Vilnius, Warsaw became the capital of the Commonwealth and the Crown of the Kingdom of...",its central location,Why did Warsaw become the capital of the Commonwealth?,"[Due to its central location, its central location, central location]","[232, 239, 243]",True
...,...,...,...,...,...,...,...,...
10201,5733d13e4776f419006612c6,French_and_Indian_War,"In 1755, six colonial governors in North America met with General Edward Braddock, the newly arrived British Army commander, and planned a four-way attack on the French. None succeeded and the main effort by Braddock was a disaster; he was defeated in the Battle of the Monongahela on July 9, 1755 and died a few days later. British operations in 1755, 1756 and 1757 in the frontier areas of Penn...","poor management, internal divisions, and effective Canadian scouts, French regular forces, and Indian warrior allies","Why did Bri

In [153]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [205]:
squad_adv_addSent.shape

{'validation': (1787, 5)}

In [225]:
for item in squad_adv_addSent['validation']
:
    print(item)


{'id': '56be4db0acb8001400a502ee', 'title': 'Super_Bowl_50', 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.', 'question': 'Where did Super Bowl 50 take place?', 'answers': {'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Fran

In [154]:
import random
random.seed(42)
squad_adv_selection = random.sample(range(squad_adv_addSent['validation'].num_rows), 500)
nq_selection = random.sample(range(3369), 400)
squad_selection = random.sample(range(10570), 500)
squad_v2_selection = random.sample(range(squad_v2['train'].num_rows), 500)

squad_adv_addSent_limited = squad_adv_addSent['validation'].select(squad_adv_selection)
nq_data_limited_unformatted = nq_data['train'].select(nq_selection)
squad_limited = squad['train'].select(squad_selection)
squad_v2_limited = squad_v2['train'].select(squad_v2_selection)

In [155]:
squad_adv_addSent['validation'][0]

{'id': '56be4db0acb8001400a502ee',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Where did Super Bowl 50 take place?',
 'answers': {'text': ['Santa Clara, California',
   "Levi's Stadium",
   "Levi's Stadium in th

In [156]:
nq_data_limited_unformatted['paragraphs'][0:1]

[[{'context': 'The Cornell Notes system (also Cornell note-taking system, Cornell method, or Cornell way) is a note-taking system devised in the 1940s by Walter Pauk, an education professor at Cornell University. Pauk advocated its use in his best-selling book How to Study in College.[1]',
   'qas': [{'answers': [{'answer_start': 139,
       'text': 'Walter Pauk, an education professor at Cornell University'},
      {'answer_start': 139, 'text': 'Walter Pauk'}],
     'id': '-5682352906951331542',
     'is_impossible': False,
     'question': 'who invented the cornell method of note taking'}]}]]

In [157]:
from datasets import Dataset
import numpy as np
ids = []
titles = []
contexts = []
questions = []
answers = []

for example in nq_data_limited_unformatted['paragraphs']:
  contexts.append(example[0]['context'])
  qas = example[0]['qas'][0]
  questions.append(qas['question'])
  #answers.append(qas['answers'])
  titles.append("natural_questioning")
  ids.append(qas['id'])
  answers_reshaped = {}

  texts = []
  answer_starts = []

  for ans in qas['answers']:
    #print("cuurent ans is: ", ans)
    if type(ans) == list:
      #print("found a list")
      for a in ans:
        #print("current a is: ",a)
        try:
          texts.append(a['text'])
          answer_starts.append(a['answer_start'])
        except:
          pass
    else:
      try:
        texts.append(ans['text'])
        answer_starts.append(ans['answer_start'])
      except:
        pass
  #print("adding: ",{"answer_start": answer_starts, "text": texts})
  answers.append({"answer_start": answer_starts, "text": texts})


my_data = {
    "id": ids,
    "title": titles,
    "context": contexts,
    "question": questions,
    "answers": answers
  
}

#print(my_data['answers'])
#print(my_data)  
nq_data_limited = Dataset.from_dict(my_data, features=squad["train"].features)


In [158]:

nq_data_limited[0]

{'id': '-5682352906951331542',
 'title': 'natural_questioning',
 'context': 'The Cornell Notes system (also Cornell note-taking system, Cornell method, or Cornell way) is a note-taking system devised in the 1940s by Walter Pauk, an education professor at Cornell University. Pauk advocated its use in his best-selling book How to Study in College.[1]',
 'question': 'who invented the cornell method of note taking',
 'answers': {'text': ['Walter Pauk, an education professor at Cornell University',
   'Walter Pauk'],
  'answer_start': [139, 139]}}

In [159]:
import numpy as np
import collections
from collections import defaultdict, OrderedDict
from transformers import Trainer, EvalPrediction
from transformers.trainer_utils import PredictionOutput
from typing import Tuple
from tqdm.auto import tqdm

QA_MAX_ANSWER_LENGTH = 30


# This function preprocesses an NLI dataset, tokenizing premises and hypotheses.
def prepare_dataset_nli(examples, tokenizer, max_seq_length=None):
    max_seq_length = tokenizer.model_max_length if max_seq_length is None else max_seq_length

    tokenized_examples = tokenizer(
        examples['premise'],
        examples['hypothesis'],
        truncation=True,
        max_length=max_seq_length,
        padding='max_length'
    )

    tokenized_examples['label'] = examples['label']
    return tokenized_examples


# This function computes sentence-classification accuracy.
# Functions with signatures like this one work as the "compute_metrics" argument of transformers.Trainer.
def compute_accuracy(eval_preds: EvalPrediction):
    return {
        'accuracy': (np.argmax(
            eval_preds.predictions,
            axis=1) == eval_preds.label_ids).astype(
            np.float32).mean().item()
    }


# This function preprocesses a question answering dataset, tokenizing the question and context text
# and finding the right offsets for the answer spans in the tokenized context (to use as labels).
# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py
def prepare_train_dataset_qa(examples, tokenizer, max_seq_length=None):
    questions = [q.lstrip() for q in examples["question"]]
    max_seq_length = tokenizer.model_max_length
    # tokenize both questions and the corresponding context
    # if the context length is longer than max_length, we split it to several
    # chunks of max_length
    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=max_seq_length,
        stride=min(max_seq_length // 2, 128),
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Since one example might give us several features if it has a long context,
    # we need a map from a feature to its corresponding example.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position
    # in the original context. This will help us compute the start_positions
    # and end_positions to get the final answer string.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        # We will label features not containing the answer the index of the CLS token.
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        # from the feature idx to sample idx
        sample_index = sample_mapping[i]
        # get the answer for a feature
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and
                    offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and \
                        offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(
                    token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


def prepare_validation_dataset_qa(examples, tokenizer):
    questions = [q.lstrip() for q in examples["question"]]
    max_seq_length = tokenizer.model_max_length
    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=max_seq_length,
        stride=min(max_seq_length // 2, 128),
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


# This function uses start and end position scores predicted by a question answering model to
# select and extract the predicted answer span from the context.
# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/utils_qa.py
def postprocess_qa_predictions(examples,
                               features,
                               predictions: Tuple[np.ndarray, np.ndarray],
                               n_best_size: int = 20):
    if len(predictions) != 2:
        raise ValueError(
            "`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(
            f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[
            example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits
            # to span of texts in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                            -1: -n_best_size - 1: -1].tolist()
            end_indexes = np.argsort(end_logits)[
                          -1: -n_best_size - 1: -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                            start_index >= len(offset_mapping)
                            or end_index >= len(offset_mapping)
                            or offset_mapping[start_index] is None
                            or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or \
                            end_index - start_index + 1 > QA_MAX_ANSWER_LENGTH:
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0],
                                        offset_mapping[end_index][1]),
                            "score": start_logits[start_index] +
                                     end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"],
                             reverse=True)[:n_best_size]

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0]: offsets[1]]

        # In the very rare edge case we have not a single non-null prediction,
        # we create a fake prediction to avoid failure.
        if len(predictions) == 0 or (
                len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0,
                                   "end_logit": 0.0, "score": 0.0})

        all_predictions[example["id"]] = predictions[0]["text"]
    return all_predictions


# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/trainer_qa.py
class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples

    def evaluate(self,
                 eval_dataset=None,  # denotes the dataset after mapping
                 eval_examples=None,  # denotes the raw dataset
                 ignore_keys=None,  # keys to be ignored in dataset
                 metric_key_prefix: str = "eval"
                 ):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        try:
            # compute the raw predictions (start_logits and end_logits)
            output = self.evaluation_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
            )
        finally:
            self.compute_metrics = compute_metrics

        if self.compute_metrics is not None:
            # post process the raw predictions to get the final prediction
            # (from start_logits, end_logits to an answer string)
            eval_preds = postprocess_qa_predictions(eval_examples,
                                                    eval_dataset,
                                                    output.predictions)
            formatted_predictions = [{"id": k, "prediction_text": v}
                                     for k, v in eval_preds.items()]
            references = [{"id": ex["id"], "answers": ex['answers']}
                          for ex in eval_examples]

            # compute the metrics according to the predictions and references
            metrics = self.compute_metrics(
                EvalPrediction(predictions=formatted_predictions,
                               label_ids=references)
            )

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)

            self.log(metrics)
        else:
            metrics = {}

        self.control = self.callback_handler.on_evaluate(self.args, self.state,
                                                         self.control, metrics)
        return metrics

In [160]:
from transformers import Trainer

In [161]:
squad_adv_addSent_limited

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 500
})

In [162]:
checklist_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 600
})

In [163]:
combined_df = datasets.concatenate_datasets([squad_limited, checklist_data])

In [164]:
# set baseline model for training start point
model_dir = "./fp-dataset-artifacts-main/squad_trained_model/"
baseline_model = AutoModelForQuestionAnswering.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
baseline_model_pipeline = pipeline('question-answering', model=baseline_model, tokenizer=tokenizer)

# Dataset selection
# In this chunk, we want to use the random subset of the adversarial squad dataset
dataset = checklist_data_full # squad_limited checklist_data combined_df  nq_data_limited squad_adv_addSent_limited

# Select the dataset preprocessing function (these functions are defined in helpers.py)
prepare_train_dataset = lambda exs: prepare_train_dataset_qa(exs, tokenizer)
prepare_eval_dataset = lambda exs: prepare_validation_dataset_qa(exs, tokenizer)


train_dataset = None
#eval_dataset = None
train_dataset_featurized = None
#eval_dataset_featurized = None

NUM_PREPROCESSING_WORKERS = 2

train_dataset = dataset
train_dataset_featurized = train_dataset.map(
    prepare_train_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=train_dataset.column_names
    )

# Select the training configuration
trainer_class = Trainer
eval_kwargs = {}
# If you want to use custom metrics, you should define your own "compute_metrics" function.
# For an example of a valid compute_metrics function, see compute_accuracy in helpers.py.
compute_metrics = None

# For QA, we need to use a tweaked version of the Trainer (defined in helpers.py)
# to enable the question-answering specific evaluation metrics
trainer_class = QuestionAnsweringTrainer
eval_kwargs['eval_examples'] = train_dataset
metric = datasets.load_metric('squad')
compute_metrics = lambda eval_preds: metric.compute(
    predictions=eval_preds.predictions, references=eval_preds.label_ids)

# This function wraps the compute_metrics function, storing the model's predictions
# so that they can be dumped along with the computed metrics
eval_predictions = None
def compute_metrics_and_store_predictions(eval_preds):
    eval_predictions = eval_preds
    return compute_metrics(eval_preds)

# Initialize the Trainer object with the specified arguments and the model and dataset we loaded above
trainer = trainer_class(
    model=baseline_model,
    train_dataset=train_dataset_featurized,
    eval_dataset=train_dataset_featurized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_and_store_predictions
)
# Train and/or evaluate

trainer.train()
trainer.save_model()
    # If you want to customize the way the loss is computed, you should subclass Trainer and override the "compute_loss"
    # method (see https://huggingface.co/transformers/_modules/transformers/trainer.html#Trainer.compute_loss).
    #
    # You can also add training hooks using Trainer.add_callback:
    #   See https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer.add_callback
    #   and https://huggingface.co/transformers/main_classes/callback.html#transformers.TrainerCallback



Map (num_proc=2):   0%|          | 0/6296 [00:00<?, ? examples/s]

/var/folders/8z/wspwslw53z54yzsmtv5qzw2h0000gn/T/ipykernel_80030/2292579616.py:42: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('squad')


You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.099100
1000,0.014900
1500,0.008300
2000,0.002500


In [182]:
pwd

'/Users/sandeepsangole/datascience/courses/nlp/final_project/fp-dataset-artifacts-main/combined_eval'

In [167]:
import pandas as pd

In [183]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_combined = pd.DataFrame(columns=columns)

with jsonlines.open('eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_combined.loc[len(answers_df_tuned_combined)] = entry
#     i+=1

In [184]:
answers_df_tuned_combined[answers_df_tuned_combined['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_combined.context.str.len() >= 0.75].groupby('exact_match').count() # 1519 / 2014=0.7542, worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,508,508,508,508,508,508,508
True,1506,1506,1506,1506,1506,1506,1506


In [185]:
answers_df_tuned_combined[answers_df_tuned_combined['question'].str.contains("Why")].groupby('exact_match').count() # 82/150=0.54666667 , worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,67,67,67,67,67,67,67
True,83,83,83,83,83,83,83


In [189]:
pwd

'/Users/sandeepsangole/datascience/courses/nlp/final_project/fp-dataset-artifacts-main/combined_eval'

In [195]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_checklist = pd.DataFrame(columns=columns)

with jsonlines.open('../tuned_eval/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_checklist.loc[len(answers_df_tuned_checklist)] = entry
#     i+=1

In [196]:
answers_df_tuned_checklist[answers_df_tuned_checklist['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_checklist.context.str.len() >= 0.75].groupby('exact_match').count() # 1520 / 2014=0.7547, worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,499,499,499,499,499,499,499
True,1515,1515,1515,1515,1515,1515,1515


In [197]:
answers_df_tuned_checklist[answers_df_tuned_checklist['question'].str.contains("Why")].groupby('exact_match').count() # 86/150=0.54666667 , worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,69,69,69,69,69,69,69
True,81,81,81,81,81,81,81


In [200]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_more_squad = pd.DataFrame(columns=columns)

with jsonlines.open('../tuned_eval/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_more_squad.loc[len(answers_df_tuned_more_squad)] = entry
#     i+=1

In [201]:
answers_df_tuned_more_squad[answers_df_tuned_more_squad['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_more_squad.context.str.len() >= 0.75].groupby('exact_match').count() # 1516 / 2014=0.7527, even worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,499,499,499,499,499,499,499
True,1515,1515,1515,1515,1515,1515,1515


In [202]:
answers_df_tuned_more_squad[answers_df_tuned_more_squad['question'].str.contains("Why")].groupby('exact_match').count() # 80 /150=0.53333 , even worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,69,69,69,69,69,69,69
True,81,81,81,81,81,81,81


In [72]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_nq = pd.DataFrame(columns=columns)

with jsonlines.open('/content/tmp_tuned_nq/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_nq.loc[len(answers_df_tuned_nq)] = entry
    i+=1

FileNotFoundError: [Errno 2] No such file or directory: '/content/tmp_tuned_nq/eval_predictions.jsonl'

In [73]:
answers_df_tuned_nq[answers_df_tuned_nq['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_nq.context.str.len() >= 0.75].groupby('exact_match').count() # 1439 / 2014=0.7144, much worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,


In [74]:
answers_df_tuned_combined

,id,title,context,pred,question,answers_written,answers_idx,exact_match


In [75]:
output_dir = "./fp-dataset-artifacts-main/improved_trained_model_v1/"
with open(os.path.join(output_dir, 'eval_metrics.json'), encoding='utf-8', mode='w') as f:
    json.dump(results, f)

    with open(os.path.join(output_dir, 'eval_predictions.jsonl'), encoding='utf-8', mode='w') as f:
        if args.task == 'qa':
            predictions_by_id = {pred['id']: pred['prediction_text'] for pred in eval_predictions.predictions}
            for example in eval_dataset:
                example_with_prediction = dict(example)
                example_with_prediction['predicted_answer'] = predictions_by_id[example['id']]
                f.write(json.dumps(example_with_prediction))
                f.write('\n')
        else:
            for i, example in enumerate(eval_dataset):
                example_with_prediction = dict(example)
                example_with_prediction['predicted_scores'] = eval_predictions.predictions[i].tolist()
                example_with_prediction['predicted_label'] = int(eval_predictions.predictions[i].argmax())
                f.write(json.dumps(example_with_prediction))
                f.write('\n')

NameError: name 'os' is not defined

In [76]:
squad_adv_addSent_limited.select([0])[0]

{'id': '572803493acd2414000df22a',
 'title': 'Doctor_Who',
 'context': 'Between about 1964 and 1973, large amounts of older material stored in the BBC\'s various video tape and film libraries were either destroyed,[note 3] wiped, or suffered from poor storage which led to severe deterioration from broadcast quality. This included many old episodes of Doctor Who, mostly stories featuring the first two Doctors: William Hartnell and Patrick Troughton. In all, 97 of 253 episodes produced during the first six years of the programme are not held in the BBC\'s archives (most notably seasons 3, 4, & 5, from which 79 episodes are missing). In 1972, almost all episodes then made were known to exist at the BBC, while by 1978 the practice of wiping tapes and destroying "spare" film copies had been brought to a stop.',
 'question': "How many of the episodes produced in the first six seasons are not in BBC's archives?",
 'answers': {'text': ['97', '79', '97'], 'answer_start': [388, 539, 388]}}

In [77]:
suite.to_raw_file('tests.jsonl', file_format="squad")

TypeError: string indices must be integers, not 'str'

In [78]:
suite.get_raw_example_list()

["('Alfred is weaker than Ashley.', 'Who is less weak?')",
 "('Alfred is weaker than Ashley.', 'Who is weaker?')",
 "('Ryan is wiser than John.', 'Who is less wise?')",
 "('Ryan is wiser than John.', 'Who is wiser?')",
 "('Brian is richer than Andrea.', 'Who is less rich?')",
 "('Brian is richer than Andrea.', 'Who is richer?')",
 "('Frances is stranger than Joseph.', 'Who is less strange?')",
 "('Frances is stranger than Joseph.', 'Who is stranger?')",
 "('Jerry is nicer than Bob.', 'Who is less nice?')",
 "('Jerry is nicer than Bob.', 'Who is nicer?')",
 "('Matthew is cooler than Jimmy.', 'Who is less cool?')",
 "('Matthew is cooler than Jimmy.', 'Who is cooler?')",
 "('Walter is stranger than Martin.', 'Who is less strange?')",
 "('Walter is stranger than Martin.', 'Who is stranger?')",
 "('Thomas is taller than Gary.', 'Who is less tall?')",
 "('Thomas is taller than Gary.', 'Who is taller?')",
 "('Sophie is taller than Ashley.', 'Who is less tall?')",
 "('Sophie is taller than Ash